## Cross Validation for Time series

This notebook includes Algorithm exploration for times series data using cross validation, 
- train-test data split
- logic for handling Imbalanced dataset
- Relevant feature transformations
- Treatment of categorical variable

This notebook also demonstrates algorithm exploration and the logic for setting up the pipeline for running the **Logistic regression model.**

### Section 1 - Setup Environment

In [0]:
from pyspark.sql.functions import col, concat, count, countDistinct, expr, lit, length, max as pyspark_max, min as pyspark_min, mean, substring, split, sum as pyspark_sum, when, to_utc_timestamp, to_timestamp, unix_timestamp, isnan
import pandas as pd
from html import escape
from IPython.display import HTML, display as ipython_display
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType, StringType, BooleanType, DateType, DoubleType
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar
from pyspark.sql.functions import substring
from pyspark.sql import functions as F
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col, max, substring
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from sparkdl.xgboost import XgboostClassifier
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import Imputer
from pyspark.ml.classification import LinearSVC as svc
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline
import numpy as np
import itertools
import time
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window


In [0]:
blob_container = "w261-container" # The name of your container created in https://portal.azure.com
storage_account = "w261storageaccount" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261scope" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

### Section 2 - Load Data

In [0]:
# Inspect the Mount's Final Project folder 
display(dbutils.fs.ls("/mnt/mids-w261/datasets_final_project"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/airlines/,airlines/,0
dbfs:/mnt/mids-w261/datasets_final_project/airlines_data/,airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/,parquet_airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0
dbfs:/mnt/mids-w261/datasets_final_project/stations_data/,stations_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/,weather_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data_6_hr/,weather_data_6_hr/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data_single/,weather_data_single/,0


In [0]:
# Load full data
full_data = True

data_df = spark.read.parquet(f"{blob_url}/full_join_route_v2/*")

display(data_df.limit(10))


year,quarter,month,day_of_week,holiday,weather_obs_prev,weather_obs_prev_2,carrier,origin,destination,departure_delay,departure_delay_15,departure_delay_boolean,arrival_delay_boolean,distance,distance_group,actual_timestamp,hourly_timestamp,prev_hourly_timestamp,utc_hourly_timestamp,utc_prev_hourly_timestamp,planned_time_between_flights,prev_dep_delayed_confirmed,weather_station,wnd_angle_prev,wnd_angle_qc_prev,wnd_type_prev,wnd_speed_prev,wnd_speed_qc_prev,wnd_cloud_angle_prev,cig_cloud_angle_qc_prev,cig_method_prev,cig_cavok_prev,vis_dist_prev,vis_dist_qc_prev,vis_var_prev,vis_var_qc_prev,tmp_c_prev,tmp_qc_prev,dew_c_prev,dew_qc_prev,slp_p_prev,slp_qc_prev,wnd_angle_prev_2,wnd_angle_qc_prev_2,wnd_type_prev_2,wnd_speed_prev_2,wnd_speed_qc_prev_2,wnd_cloud_angle_prev_2,cig_cloud_angle_qc_prev_2,cig_method_prev_2,cig_cavok_prev_2,vis_dist_prev_2,vis_dist_qc_prev_2,vis_var_prev_2,vis_var_qc_prev_2,tmp_c_prev_2,tmp_qc_prev_2,dew_c_prev_2,dew_qc_prev_2,slp_p_prev_2,slp_qc_prev_2,route_delay,origin_arrival_delay,origin_departure_delay,dest_departure_delay,dest_arrival_delay
2019,1,2,7,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-02-10T10:38:00.000+0000,2019-02-10T08:00:00.000+0000,2019-02-10T07:00:00.000+0000,2019-02-10T13:00:00.000+0000,2019-02-10T12:00:00.000+0000,45.0,0.0,72518014735,310,5,N,21,5,1219,5,M,N,16093,5,N,5,-44,5,-122,5,10384,5,300,5,N,31,5,1311,5,M,N,16093,5,N,5,-50,5,-122,5,10385,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,1,3,7,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-03-31T10:18:00.000+0000,2019-03-31T08:00:00.000+0000,2019-03-31T07:00:00.000+0000,2019-03-31T12:00:00.000+0000,2019-03-31T11:00:00.000+0000,45.0,0.0,72518014735,180,5,N,88,5,2134,5,M,N,16093,5,N,5,144,5,72,5,10017,5,170,5,N,103,5,2591,5,M,N,16093,5,N,5,150,5,72,5,10017,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,2,6,7,0,0,0,G4,ALB,PGD,39.0,1.0,1.0,1.0,1185.0,5,2019-06-16T11:37:00.000+0000,2019-06-16T09:00:00.000+0000,2019-06-16T08:00:00.000+0000,2019-06-16T13:00:00.000+0000,2019-06-16T12:00:00.000+0000,45.0,0.0,72518014735,170,5,N,26,5,914,5,M,N,9656,5,N,5,189,5,167,5,10110,5,180,5,N,26,5,853,5,M,N,16093,5,N,5,194,5,167,5,10111,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,2,6,4,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-06-20T11:37:00.000+0000,2019-06-20T09:00:00.000+0000,2019-06-20T08:00:00.000+0000,2019-06-20T13:00:00.000+0000,2019-06-20T12:00:00.000+0000,45.0,0.0,72518014735,170,5,N,46,5,335,5,M,N,16093,5,N,5,217,5,183,5,10022,5,160,5,N,41,5,274,5,M,N,16093,5,N,5,217,5,183,5,10029,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,1,1,2,1,0,0,G4,ALB,PGD,57.0,1.0,1.0,1.0,1185.0,5,2019-01-01T18:40:00.000+0000,2019-01-01T16:00:00.000+0000,2019-01-01T15:00:00.000+0000,2019-01-01T21:00:00.000+0000,2019-01-01T20:00:00.000+0000,45.0,1.0,72518014735,300,5,N,67,5,792,5,M,N,16093,5,N,5,33,5,-28,5,10172,5,300,5,N,51,5,792,5,M,N,16093,5,N,5,39,5,-28,5,10160,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,3,8,7,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-08-18T10:48:00.000+0000,2019-08-18T08:00:00.000+0000,2019-08-18T07:00:00.000+0000,2019-08-18T12:00:00.000+0000,2019-08-18T11:00:00.000+0000,45.0,0.0,72518014735,null,null,C,0,5,122,5,M,N,3219,5,N,5,206,5,189,5,10171,5,160,5,N,21,5,122,5,M,N,1609,5,N,5,200,5,183,5,10168,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,2,5,4,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,1.0,1185.0,5,2019-05-09T16:45:00.000+0000,2019-05-09T14:00:00.000+0000,2019-05-09T13:00:00.000+0000,2019-05-09T18:00:00.000+0000,2019-05-09T17:00:00.000+0000,45.0,0.0,72518014735,170,5,V,57,5,1676,5,M,N,16093,5,N,5,189,5,22,5,10229,5,170,5,N,51,5,2438,5,M,N,16093,5,N,5,178,5,11,5,10242,5,14.129032258064516,11.878245986779982,11.6658600

### Section 3 - Define helper functions

In [0]:
# Helper function 1:
def missing_values(helper_df, sort=False):
  """
  Helper function to count number of nulls and nans in each column
  """
  df = helper_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in helper_df.dtypes if c_type not in ('timestamp')]).toPandas()
  
  if len(df) == 0:
    print("There are no missing values")
    return None
  
  if sort:
    return df.rename(index={0: 'count'}).T.sort_values("count", ascending=False)
  
  return df


In [0]:
# Helper function 2

def main():

    global eval_metrics

# f1 = 0.0
# recall = 0.0
# precision = 0.0
# sensitivity = 0.0
# specificity = 0.0
# fp = 0.0
# fn = 0.0

# metrics = (f1, recall, precision, sensitivity, specificity, fp, fn)



def print_results(predictions, model_name): 
  """
  Helper function to print evaluation metrics
  """
  tp = predictions[(predictions.departure_delay_boolean == 1) & (predictions.prediction == 1)].count()
  tn = predictions[(predictions.departure_delay_boolean == 0) & (predictions.prediction == 0)].count()
  fp = predictions[(predictions.departure_delay_boolean == 0) & (predictions.prediction == 1)].count()
  fn = predictions[(predictions.departure_delay_boolean == 1) & (predictions.prediction == 0)].count()
  total = predictions.count()
  
  recall = float(tp)/(tp+fn)
  precision = float(tp)/(tp+fp)
  f1 = (2*recall*precision)/(precision+recall)
  
  data = {'Actual-delay': [tp, fn], 'Actual-on time': [fp, tn]}
  confusion_matrix = pd.DataFrame.from_dict(data, orient='index', columns=['Predicted-delay', 'Predicted-on time'])
  
  #print("Test Area Under ROC: ", "{:.2f}".format(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'})))
  #print("Test Area Under Precision-Recall Curve: ", "{:.2f}".format(evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderPR'})))

  print("Sensitivity: {:.2%}".format(tp/(tp + fn)))
  print("Specificity: {:.2%}".format(tn/(tn + fp)))
  print("False positive rate: {:.2%}".format(fp/(fp + tn)))
  print("False negative rate: {:.2%}".format(fn/(tp + fn)))
  print("Recall: {:.2%}".format(recall))
  print("Precision: {:.2%}".format(precision))
  print("f1: {:.2%}".format(f1))
  
  metrics = (round(f1, 2), round(recall, 2), round(precision, 2), round(tp/(tp + fn), 2), \
             round(tn/(tn + fp), 2), round(fp/(fp + tn), 2), round(fn/(tp + fn), 2))
  
  global eval_metrics
  eval_metrics = {model_name: metrics}
  
  print("########### Confusion Matrix ###########")
  print(confusion_matrix)

  #print(eval_metrics)

In [0]:
def get_results(predictions, model_name): 
  """
  Helper function to print evaluation metrics
  """
  tp = predictions[(predictions.departure_delay_boolean == 1) & (predictions.prediction == 1)].count()
  tn = predictions[(predictions.departure_delay_boolean == 0) & (predictions.prediction == 0)].count()
  fp = predictions[(predictions.departure_delay_boolean == 0) & (predictions.prediction == 1)].count()
  fn = predictions[(predictions.departure_delay_boolean == 1) & (predictions.prediction == 0)].count()
  total = predictions.count()
  
  recall = float(tp)/(tp+fn)
  precision = float(tp)/(tp+fp)
  f1 = (2*recall*precision)/(precision+recall)
  
  return round(f1, 2), round(precision, 2), round(recall, 2)
  

### Section 4 - Feature Engineering

In [0]:
# Create a feature to include the hour of the day in local time
from pyspark.sql.functions import hour

data_df = data_df.withColumn('hour',hour(data_df.actual_timestamp))

display(data_df.limit(10))


year,quarter,month,day_of_week,holiday,weather_obs_prev,weather_obs_prev_2,carrier,origin,destination,departure_delay,departure_delay_15,departure_delay_boolean,arrival_delay_boolean,distance,distance_group,actual_timestamp,hourly_timestamp,prev_hourly_timestamp,utc_hourly_timestamp,utc_prev_hourly_timestamp,planned_time_between_flights,prev_dep_delayed_confirmed,weather_station,wnd_angle_prev,wnd_angle_qc_prev,wnd_type_prev,wnd_speed_prev,wnd_speed_qc_prev,wnd_cloud_angle_prev,cig_cloud_angle_qc_prev,cig_method_prev,cig_cavok_prev,vis_dist_prev,vis_dist_qc_prev,vis_var_prev,vis_var_qc_prev,tmp_c_prev,tmp_qc_prev,dew_c_prev,dew_qc_prev,slp_p_prev,slp_qc_prev,wnd_angle_prev_2,wnd_angle_qc_prev_2,wnd_type_prev_2,wnd_speed_prev_2,wnd_speed_qc_prev_2,wnd_cloud_angle_prev_2,cig_cloud_angle_qc_prev_2,cig_method_prev_2,cig_cavok_prev_2,vis_dist_prev_2,vis_dist_qc_prev_2,vis_var_prev_2,vis_var_qc_prev_2,tmp_c_prev_2,tmp_qc_prev_2,dew_c_prev_2,dew_qc_prev_2,slp_p_prev_2,slp_qc_prev_2,route_delay,origin_arrival_delay,origin_departure_delay,dest_departure_delay,dest_arrival_delay,hour
2019,1,2,7,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-02-10T10:38:00.000+0000,2019-02-10T08:00:00.000+0000,2019-02-10T07:00:00.000+0000,2019-02-10T13:00:00.000+0000,2019-02-10T12:00:00.000+0000,45.0,0.0,72518014735,310,5,N,21,5,1219,5,M,N,16093,5,N,5,-44,5,-122,5,10384,5,300,5,N,31,5,1311,5,M,N,16093,5,N,5,-50,5,-122,5,10385,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844,10
2019,1,3,7,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-03-31T10:18:00.000+0000,2019-03-31T08:00:00.000+0000,2019-03-31T07:00:00.000+0000,2019-03-31T12:00:00.000+0000,2019-03-31T11:00:00.000+0000,45.0,0.0,72518014735,180,5,N,88,5,2134,5,M,N,16093,5,N,5,144,5,72,5,10017,5,170,5,N,103,5,2591,5,M,N,16093,5,N,5,150,5,72,5,10017,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844,10
2019,2,6,7,0,0,0,G4,ALB,PGD,39.0,1.0,1.0,1.0,1185.0,5,2019-06-16T11:37:00.000+0000,2019-06-16T09:00:00.000+0000,2019-06-16T08:00:00.000+0000,2019-06-16T13:00:00.000+0000,2019-06-16T12:00:00.000+0000,45.0,0.0,72518014735,170,5,N,26,5,914,5,M,N,9656,5,N,5,189,5,167,5,10110,5,180,5,N,26,5,853,5,M,N,16093,5,N,5,194,5,167,5,10111,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844,11
2019,2,6,4,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-06-20T11:37:00.000+0000,2019-06-20T09:00:00.000+0000,2019-06-20T08:00:00.000+0000,2019-06-20T13:00:00.000+0000,2019-06-20T12:00:00.000+0000,45.0,0.0,72518014735,170,5,N,46,5,335,5,M,N,16093,5,N,5,217,5,183,5,10022,5,160,5,N,41,5,274,5,M,N,16093,5,N,5,217,5,183,5,10029,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844,11
2019,1,1,2,1,0,0,G4,ALB,PGD,57.0,1.0,1.0,1.0,1185.0,5,2019-01-01T18:40:00.000+0000,2019-01-01T16:00:00.000+0000,2019-01-01T15:00:00.000+0000,2019-01-01T21:00:00.000+0000,2019-01-01T20:00:00.000+0000,45.0,1.0,72518014735,300,5,N,67,5,792,5,M,N,16093,5,N,5,33,5,-28,5,10172,5,300,5,N,51,5,792,5,M,N,16093,5,N,5,39,5,-28,5,10160,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844,18
2019,3,8,7,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,0.0,1185.0,5,2019-08-18T10:48:00.000+0000,2019-08-18T08:00:00.000+0000,2019-08-18T07:00:00.000+0000,2019-08-18T12:00:00.000+0000,2019-08-18T11:00:00.000+0000,45.0,0.0,72518014735,null,null,C,0,5,122,5,M,N,3219,5,N,5,206,5,189,5,10171,5,160,5,N,21,5,122,5,M,N,1609,5,N,5,200,5,183,5,10168,5,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844,10
2019,2,5,4,0,0,0,G4,ALB,PGD,0.0,0.0,0.0,1.0,1185.0,5,2019-05-09T16:45:00.000+0000,2019-05-09T14:00:00.000+0000,2019-05-09T13:00:00.000+0000,2019-05-09T18:00:00.000+0000,2019-05-09T17:00:00.000+0000,45.0,0.0,72518014735,170,5,V,57,5,1676,5,M,N,16093,5,N,5,189,5,22,5,10229,5,170,5,N,51,5,2438,5,M,N,16093,5,N,5,178,5,11,5,10242,5,14.129032258064516,11.878

**Feature Selection**

In [0]:

#Select ML Vars - Variables for building models
split_data_df = data_df.selectExpr(
                              "cast (year as string) year",
                              "cast (month as string) month",
                              "cast (day_of_week as string) day_of_week",
                              "actual_timestamp",
                              "cast (hour as string) hour",
                              "carrier",
                              "holiday",
                              "weather_obs_prev",
                              "weather_obs_prev_2",
                              "origin",
                              "destination",
                              "departure_delay_boolean",
                              "planned_time_between_flights",
                              "cast (prev_dep_delayed_confirmed as string) prev_dep_delayed_confirmed",    
                              "distance",
                              "cast (wnd_angle_prev as int) wnd_angle_prev",                               
                              "wnd_type_prev",
                              "wnd_speed_prev_2",
                              "wnd_speed_prev",
                              "wnd_cloud_angle_prev",
                              "vis_dist_prev",
                              "vis_dist_prev_2",
                              "vis_var_prev",
                              "tmp_c_prev",
                              "dew_c_prev",
                              "route_delay",
                              "origin_arrival_delay",
                              "origin_departure_delay",
                              "dest_departure_delay",
                              "dest_arrival_delay",
                             )

display(split_data_df.limit(10))


year,month,day_of_week,actual_timestamp,hour,carrier,holiday,weather_obs_prev,weather_obs_prev_2,origin,destination,departure_delay_boolean,planned_time_between_flights,prev_dep_delayed_confirmed,distance,wnd_angle_prev,wnd_type_prev,wnd_speed_prev_2,wnd_speed_prev,wnd_cloud_angle_prev,vis_dist_prev,vis_dist_prev_2,vis_var_prev,tmp_c_prev,dew_c_prev,route_delay,origin_arrival_delay,origin_departure_delay,dest_departure_delay,dest_arrival_delay
2019,2,7,2019-02-10T10:38:00.000+0000,10,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,310,N,31,21,1219,16093,16093,N,-44,-122,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,3,7,2019-03-31T10:18:00.000+0000,10,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,180,N,103,88,2134,16093,16093,N,144,72,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,6,7,2019-06-16T11:37:00.000+0000,11,G4,0,0,0,ALB,PGD,1.0,45.0,0.0,1185.0,170,N,26,26,914,9656,16093,N,189,167,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,6,4,2019-06-20T11:37:00.000+0000,11,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,170,N,41,46,335,16093,16093,N,217,183,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,1,2,2019-01-01T18:40:00.000+0000,18,G4,1,0,0,ALB,PGD,1.0,45.0,1.0,1185.0,300,N,51,67,792,16093,16093,N,33,-28,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,8,7,2019-08-18T10:48:00.000+0000,10,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,null,C,21,0,122,3219,1609,N,206,189,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,5,4,2019-05-09T16:45:00.000+0000,16,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,170,V,51,57,1676,16093,16093,N,189,22,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,1,4,2019-01-17T10:38:00.000+0000,10,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,20,N,0,15,22000,16093,16093,N,-106,-194,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,11,7,2019-11-03T10:48:00.000+0000,10,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,170,N,21,21,22000,16093,16093,N,56,0,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844
2019,3,3,2019-03-06T10:08:00.000+0000,10,G4,0,0,0,ALB,PGD,0.0,45.0,0.0,1185.0,290,N,26,36,22000,16093,16093,N,-89,-150,14.129032258064516,11.878245986779982,11.66586008813346,18.286828181565024,19.56242457996844


#### 4.1. Split Data

In [0]:
### Add a cumulative rank to dataframe and then split (80-20) for time series data.

if(full_data):

  split_data_df = split_data_df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("actual_timestamp")))

# display(split_data_df)


In [0]:
# timeseries split of train data for cross validation

if(full_data):

 # train_df = split_data_df.where("rank <= .8").drop("rank","actual_timestamp")
  train_df1 = split_data_df.where("rank <= .2").drop("rank","actual_timestamp")
  train_df2 = split_data_df.where("rank <= .4").drop("rank","actual_timestamp")
  train_df3 = split_data_df.where("rank <= .6").drop("rank","actual_timestamp")
  train_df4 = split_data_df.where("rank <= .8").drop("rank","actual_timestamp")
 #   display(train_df)

#   print(f'The train dataset has {train_df.count()} rows')

In [0]:

# timeseries split of train data for cross validation

if(full_data):
  #test_df = split_data_df.where("rank > .8").drop("rank", "actual_timestamp")
  test_df1 = split_data_df.where("rank > .2" and "rank <= 0.4").drop("rank", "actual_timestamp")
  test_df2 = split_data_df.where("rank > .4" and "rank <= 0.6").drop("rank", "actual_timestamp")
  test_df3 = split_data_df.where("rank > .6" and "rank <= 0.8").drop("rank", "actual_timestamp")
  test_df4 = split_data_df.where("rank > .8").drop("rank", "actual_timestamp")

    

In [0]:
# creating list for storing all training and test dataframes for separate windows of cross validation
train_df_list = [train_df1, train_df2, train_df3, train_df4]
test_df_list = [test_df1, test_df2, test_df3, test_df4]


In [0]:
# Let's verify if train data has any nulls

missing_values_df_train = missing_values(train_df1)

missing_values_df2 = pd.DataFrame({
                 "features": list(missing_values_df_train.columns),
                 "number_of_nulls": list(missing_values_df_train.iloc[0])})

missing_values_df2

Out[49]:

,features,number_of_nulls
0,year,0
1,month,0
2,day_of_week,0
3,hour,0
4,carrier,0
5,holiday,0
6,weather_obs_prev,6887
7,weather_obs_prev_2,11507
8,origin,0
9,destination,0


#### 4.2. Handling Imbalanced data - Undersampling

We used undersampling for handling imbalance in data since we have a very large dataset. 
SMOTE is another method to handle imbalanced data and has been implemented in a separate notebook located at: https://adb-6759024569771990.10.azuredatabricks.net/?o=6759024569771990#notebook/671119174920056/command/671119174920058

In [0]:
# Display proportion of labeled class
  
def get_proportion_labels(df):
  '''Function to display proportion of labeled class'''
  if verbose:
    print("In method - get_proportion_labels - displaying proportion of labeled class")
    print(display(df.groupby('departure_delay_boolean').count()))
  
  positive = df.filter(df.departure_delay_boolean == 1).count()
  negative = df.filter(df.departure_delay_boolean == 0).count()
  total = negative + positive
  if total == 0:
    raise Exception("No records found!")
  
  if positive == 0:
    raise Exception("No positive records found!")
  
  if negative == 0:
    raise Exception("No negative records found!")
    
  # there is a risk that the positive/negative classes are so imbalanced that they are non existent in the df
  # so we should guard against that case in order to avoid throwing div by 0
  np = -1 if positive == 0 else 1.0 * negative/positive
  pn = -1 if negative == 0 else 1.0 * positive/negative
  
  return 1.0 * positive/total, 1.0 * negative/total, pn, np

In [0]:
# downsample majority class dataset (i.e on-time flight data) to balance and match minority class dataset(i.e. delayed flights)

def downsample(df, min_major_class_ratio, alpha=0.99):
  '''Function to implement undersampling logic'''
  if min_major_class_ratio == -1:
    # assign default value to reduce the majority class by half
    min_major_class_ratio = 0.5
    print("In method downsample: Warning - reset min_major_class_ratio to default: {}".format(min_major_class_ratio))
    
  if verbose:
    print("Starting to downsample, negative class has {} rows and positive class has {} rows".format(df.filter(df.departure_delay_boolean == 0).count(), df.filter(df.departure_delay_boolean == 1).count()))
    
  negative = df.filter(df.departure_delay_boolean == 0).sample(False, min_major_class_ratio * alpha, seed=2021)
  positive = df.filter(df.departure_delay_boolean == 1)
  
  new_df = positive.union(negative).cache()
  if verbose:
    negative = new_df.filter(new_df.departure_delay_boolean ==0).count()
    positive = new_df.filter(new_df.departure_delay_boolean ==1).count()
    print("After downsampling, negative class has {} rows and positive class has {} rows".format(negative, positive))
  
  return new_df


In [0]:
# Assigning downsampled training dataset to be used further for modeling
train_df_downsampled_list = []
for index in range(len(train_df_list)):
  verbose = True
  pt, nt, pn, np = get_proportion_labels(train_df_list[index])
  train_df_downsampled = downsample(train_df_list[index], pn)
  train_df_downsampled_list.append(train_df_downsampled)


In method - get_proportion_labels - displaying proportion of labeled class

departure_delay_boolean,count
0.0,5241720
1.0,1266116


None
Starting to downsample, negative class has 5241720 rows and positive class has 1266116 rows
After downsampling, negative class has 1253874 rows and positive class has 1266116 rows
In method - get_proportion_labels - displaying proportion of labeled class

departure_delay_boolean,count
0.0,10557383
1.0,2458315


None
Starting to downsample, negative class has 10557383 rows and positive class has 2458315 rows
After downsampling, negative class has 2437088 rows and positive class has 2458315 rows
In method - get_proportion_labels - displaying proportion of labeled class

departure_delay_boolean,count
0.0,15812582
1.0,3708943
null,2011


None
Starting to downsample, negative class has 15812582 rows and positive class has 3708943 rows
After downsampling, negative class has 3674682 rows and positive class has 3708943 rows
In method - get_proportion_labels - displaying proportion of labeled class

departure_delay_boolean,count
0.0,21008863
1.0,5017444
null,5032


None
Starting to downsample, negative class has 21008863 rows and positive class has 5017444 rows
After downsampling, negative class has 4969137 rows and positive class has 5017444 rows

In [0]:
# writing all 4 downsampled train dataframes from cross validation set
train_df_downsampled_list[0].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_cv_02")
train_df_downsampled_list[1].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_cv_04")
train_df_downsampled_list[2].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_cv_06")
train_df_downsampled_list[3].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_cv_08")


In [0]:
# writing all 4 test dataframes from cross validation set
test_df_list[0].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_cv_02")
test_df_list[1].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_cv_04")
test_df_list[2].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_cv_06")
test_df_list[3].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_cv_08")

In [0]:
print("Checking downsample, negative class has {} rows and positive class has {} rows".format(train_df_downsampled_list[0].filter(train_df_downsampled_list[0].departure_delay_boolean == 0).count(), train_df_downsampled_list[0].filter(train_df_downsampled_list[0].departure_delay_boolean == 1).count()))


Checking downsample, negative class has 1253874 rows and positive class has 1266116 rows

In [0]:
# Reading train and test data
#'''
train_df_downsampled_list = []
train_df_downsampled_list.append(spark.read.parquet(f"{blob_url}/model_train_data_cv_02/*"))
train_df_downsampled_list.append(spark.read.parquet(f"{blob_url}/model_train_data_cv_04/*"))
train_df_downsampled_list.append(spark.read.parquet(f"{blob_url}/model_train_data_cv_06/*"))
train_df_downsampled_list.append(spark.read.parquet(f"{blob_url}/model_train_data_cv_08/*"))

test_df_list = []
test_df_list.append(spark.read.parquet(f"{blob_url}/model_test_data_cv_02/*"))
test_df_list.append(spark.read.parquet(f"{blob_url}/model_test_data_cv_04/*"))
test_df_list.append(spark.read.parquet(f"{blob_url}/model_test_data_cv_06/*"))
test_df_list.append(spark.read.parquet(f"{blob_url}/model_test_data_cv_08/*"))

#'''

In [0]:
print("Checking downsample, negative class has {} rows and positive class has {} rows".format(train_df_downsampled_list[0].filter(train_df_downsampled_list[0].departure_delay_boolean == 0).count(), train_df_downsampled_list[0].filter(train_df_downsampled_list[0].departure_delay_boolean == 1).count()))

Checking downsample, negative class has 1253874 rows and positive class has 1266116 rows

In [0]:
#Preparing final dataset for modeling

from pyspark.ml.feature import (OneHotEncoder, StringIndexer)
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# Let's find out the datatype of the features, which will be the same for both train and test data
model_data_train_list = []
model_data_test_list = []

for index in range(len(train_df_downsampled_list)):
  print("index", index)
  train_df = train_df_downsampled_list[index]
  test_df = test_df_list[index]
  print("\n----Categorical columns train-----")
  cat_cols = [x for (x, dataType) in train_df.dtypes if dataType == "string"]
  print(cat_cols)

  print("\n----Numeric columns train-----")
  num_cols = [x for (x, dataType) in train_df.dtypes if (((dataType == "double") or (dataType == "int")) & (x != "departure_delay_boolean"))]
  print(num_cols)
  
  # Changes nulls to 9* for cat vars, not have an impact to the rest of the data since one-hot encoding
  train_df = train_df.fillna('-999', subset=cat_cols)
  test_df = test_df.fillna('-999', subset=cat_cols)


  string_indexer = [
      StringIndexer(inputCol=x, outputCol=x + "_StringIndexer", handleInvalid="skip")
      for x in cat_cols
  ]

  one_hot_encoder = [
      OneHotEncoder(
          inputCols=[f"{x}_StringIndexer" for x in cat_cols],
          outputCols=[f"{x}_OneHotEncoder" for x in cat_cols],
      )
  ]

  imputer = [
        Imputer(inputCol=x, outputCol= x + "_Imputed").setStrategy("median")
        for x in num_cols
  ]

  scaler_vector_assembler = VectorAssembler(inputCols = [f"{x}_Imputed" for x in num_cols], outputCol="numeric_vec")
  scaler_vector_assembler.setHandleInvalid('keep')

  scaler = StandardScaler(inputCol="numeric_vec", outputCol="scaled_features")
  #Output vector assembler to be used at the modeling step, 

  assembler_input = ["scaled_features"]
  assembler_input += [f"{x}_OneHotEncoder" for x in cat_cols]

  vector_assembler = VectorAssembler(
      inputCols=assembler_input, outputCol="VectorAssembler_features"
  )

  # Pipeline Stages
  stages = []
  stages += imputer
  stages += [scaler_vector_assembler , scaler]
  stages += string_indexer
  stages += one_hot_encoder
  stages += [vector_assembler]

  # Setup Pipeline using the stages and train & test data
  general_pipeline = Pipeline().setStages(stages)
  pipeline_model = general_pipeline.fit(train_df)
  model_data_test = pipeline_model.transform(test_df)
  model_data_train = pipeline_model.transform(train_df) 

  # label = departure_delay_boolean, features = VectorAssembler_features
  selectedcols = ["departure_delay_boolean", "VectorAssembler_features"]
  model_data_train = model_data_train.select(selectedcols)
  model_data_test = model_data_test.select(selectedcols)
  
  model_data_train_list.append(model_data_train)
  model_data_test_list.append(model_data_test)


index 0

----Categorical columns train-----
['year', 'month', 'day_of_week', 'hour', 'carrier', 'holiday', 'weather_obs_prev', 'weather_obs_prev_2', 'origin', 'destination', 'prev_dep_delayed_confirmed', 'wnd_type_prev', 'vis_var_prev']

----Numeric columns train-----
['planned_time_between_flights', 'distance', 'wnd_angle_prev', 'wnd_speed_prev_2', 'wnd_speed_prev', 'wnd_cloud_angle_prev', 'vis_dist_prev', 'vis_dist_prev_2', 'tmp_c_prev', 'dew_c_prev', 'route_delay', 'origin_arrival_delay', 'origin_departure_delay', 'dest_departure_delay', 'dest_arrival_delay']
index 1

----Categorical columns train-----
['year', 'month', 'day_of_week', 'hour', 'carrier', 'holiday', 'weather_obs_prev', 'weather_obs_prev_2', 'origin', 'destination', 'prev_dep_delayed_confirmed', 'wnd_type_prev', 'vis_var_prev']

----Numeric columns train-----
['planned_time_between_flights', 'distance', 'wnd_angle_prev', 'wnd_speed_prev_2', 'wnd_speed_prev', 'wnd_cloud_angle_prev', 'vis_dist_prev', 'vis_dist_prev_2', 'tmp_c_prev', 'dew_c_prev', 'route_delay', 'origin_arrival_delay', 'origin_departure_delay', 'dest_departure_delay', 'dest_arrival_delay']
index 2

----Categorical columns train-----
['year', 'month', 'day_of_week', 'hour', 'carrier', 'holiday', 'weather_obs_prev', 'weather_obs_prev_2', 'origin', 'destination', 'prev_dep_delayed_confirmed', 'wnd_type_prev', 'vis_var_prev']

----Numeric columns train-----
['planned_time_between_flights', 'distance', 'wnd_angle_prev', 'wnd_speed_prev_2', 'wnd_speed_prev', 'wnd_cloud_angle_prev', 'vis_dist_prev', 'vis_dist_prev_2', 'tmp_c_prev', 'dew_c_prev', 'route_delay', 'origin_arrival_delay', 'origin_departure_delay', 'dest_departure_delay', 'dest_arrival_delay']
index 3

----Categorical columns train-----
['year', 'month', 'day_of_week', 'hour', 'carrier', 'holiday', 'weather_obs_prev', 'weather_obs_prev_2', 'origin', 'destination', 'prev_dep_delayed_confirmed', 'wnd_type_prev', 'vis_var_prev']

----Numeric columns train-----
['planned_time_between_flights', 'distance', 'wnd_angle_prev', 'wnd_speed_prev_2', 'wnd_speed_prev', 'wnd_cloud_angle_prev', 'vis_dist_prev', 'vis_dist_prev_2', 'tmp_c_prev', 'dew_c_prev', 'route_delay', 'origin_arrival_delay', 'origin_departure_delay', 'dest_departure_delay', 'dest_arrival_delay']

In [0]:
# writing all 4 train dataframes 
model_data_train_list[0].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_list_02")
model_data_train_list[1].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_list_04")
model_data_train_list[2].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_list_06")
model_data_train_list[3].write.mode('overwrite').parquet(f"{blob_url}/model_train_data_list_08")


In [0]:
# writing all 4 test dataframes
model_data_test_list[0].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_list_02")
model_data_test_list[1].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_list_04")
model_data_test_list[2].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_list_06")
model_data_test_list[3].write.mode('overwrite').parquet(f"{blob_url}/model_test_data_list_08")

In [0]:
# Reading model train and test data
#'''
model_data_train_list = []
model_data_train_list.append(spark.read.parquet(f"{blob_url}/model_train_data_list_02/*"))
model_data_train_listt.append(spark.read.parquet(f"{blob_url}/model_train_data_list_04/*"))
model_data_train_list.append(spark.read.parquet(f"{blob_url}/model_train_data_list_06/*"))
model_data_train_list.append(spark.read.parquet(f"{blob_url}/model_train_data_list_08/*"))

model_data_test_list = []
model_data_test_list.append(spark.read.parquet(f"{blob_url}/model_test_data_list_02/*"))
model_data_test_list.append(spark.read.parquet(f"{blob_url}/model_test_data_list_04/*"))
model_data_test_list.append(spark.read.parquet(f"{blob_url}/model_test_data_list_06/*"))
model_data_test_list.append(spark.read.parquet(f"{blob_url}/model_test_data_list_08/*"))

#'''

### Section 5 - ML Section

#### 5.1. Logistic regression

**Logistic Regression Pipeline**

In [0]:
# Setup pipeline for Logistic Regression

from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

start_lr = time.time()

lr = LogisticRegression(featuresCol='VectorAssembler_features',labelCol='departure_delay_boolean', regParam = 0.2, fitIntercept=True, elasticNetParam=0.0, maxIter=10, threshold = 0.6)

stages_lr = [lr]
pipeline_lr = Pipeline(stages=stages_lr)

classification_report_list = []
confusion_matrix_list = []
lrpredictions_list = []
for index in range(len(model_data_train_list)):
  print("index", index)
  model_data_train = model_data_train_list[index]
  mode_data_test = model_data_test_list[index]
  lr_model = pipeline_lr.fit(model_data_train)
  lrPredictions = lr_model.transform(model_data_test)
  print(f'lr model completed job in {time.time() - start_lr} seconds.')
  lr_time_model = time.time() - start_lr
  
  # Model Evaluation with scikit learn
  y_true_lr = lrPredictions.select(['departure_delay_boolean']).collect()
  y_pred_lr = lrPredictions.select(['prediction']).collect()

  lrpredictions_list.append(lrpredictions)
  
f1_score_list = []
for index in range(len(lrpredictions_list)):
  result = get_results(lrpredictions_list[index])
  f1_score_list.append(result[0])
  
avg_f1_score = sum(f1_score_list)/len(f1_score_list)

**Logistic Regression Model Evaluation**

In [0]:
# Print results with helper function

print_results(lrPredictions, 'LR')